# A UI for managing the inventory of a library of things
By Pablo
(UvA Data Processing 2025)

A library of things works like a library, but any object can be borrowed. Think of objects you will need to use at some point, but only for a short while. For example: a drill. I want to set one up eventually in my life as a small community initiative run by volunteers. If the thing becomes big enough, a digital database of the people will come in very handy. Because not everyone is acquainted with SQL, a UI would be necessary.

My initial idea was to make the database & UI record (besides items and people) item ownership, item location, item tags, item tag categories, a vouching system, and a detailed log of events. The UML diagram for the database looked like this: ![structure_old](notebook_images/structure_old.jpg)

Once I started working on the project, I realised I had to make it more humble, so it ended up looking more like this: ![structure_old](notebook_images/structure_new.jpg)

In this project, I built a UI with the python library [tkinter](https://docs.python.org/3/library/tkinter.html). It allows the user to add / delete people and items, edit their fields (except ID), and edit an item's ownership and location. Any of these actions is logged, and the user can view the list of all logged actions, but not edit it.

## The basic structure

My code relies on two files. A database file I generated beforehand following the UML diagram above, and a python file that initiates the UI. The python file:
1. Imports the necessary libraries
2. Connects to the database and opens a cursor to interact with it
3. Defines some basic functions to be used across menus
4. Defines a function to open a window for each of these menus:
    1. The main menu, where other menus can be opened
    2. The items menu
    3. The people menu
    4. The list of logged events
 5. Runs the main function, which runs the main menu function

```python
import sqliteimport sqlite`3
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from datetime import datetime

con = sqlite3.connect("library.db", timeout = 5) # connect to database
cur = con.cursor() # create cursor for executing SQL code

def main():

    main_menu()
    con.close() # close connection to database

    
def log_event(...)
    
def add_person(...)
    
def add_item(...)

def remove(...)

def edit_ownership(...)

def get_datum(...)

def get_item_data(...)

def get_person_data(...)

def get_item_data((...))


def main_menu()

def items_menu()

def people_menu()

def event_logs()


if __name__ == '__main__':
    main()
```

Because of my limited time in this presentation, I cannot go over every single function. I will however go over some that will hopefully exemplify the interactions between tkinter, sqlite3, and python at play.

## The main menu
![mainmenu](notebook_images/mainmenu.png)

The main menu sets up a few simple widgets: a label with a title, and three buttons, each calling a function that opens another window to their respective sub-menus.

```python
def main_menu():
    # set up root window
    root = tk.Tk()
    root.title("Main menu")
    root.geometry("400x300")

    # set up frame within the window. Good practice to avoid graphical bugs across operating systems.
    rootframe = ttk.Frame(root, padding="10")
    rootframe.grid(column=0, row=0, sticky="nwes")
    root.columnconfigure(0, weight=1)
    root.rowconfigure(0, weight=1)

    # set up each button
    ttk.Label(rootframe, text="Library of things", anchor="center", font=("Helvetica", 30)).grid(row=0)
    ttk.Button(rootframe, text="Items", command=items_menu).grid(row=1)
    ttk.Button(rootframe, text="People", command=people_menu).grid(row=2)
    ttk.Button(rootframe, text="History of changes", command=event_logs).grid(row=3)

    rootframe.grid_columnconfigure(0, weight=1) # tell frame it can expand within the window
    for i in range(4): # tell columns they can expand within the frame
        rootframe.grid_rowconfigure(i, weight=1)
    for child in rootframe.winfo_children(): # tell widgets they can expand within columns
        child.grid_configure(padx=5, pady=5, sticky="nsew")

    root.mainloop() # open window
```

## The items menu
![itemsmenu](notebook_images/itemsmenu.png)

There are a few things going on in the items_menu() function. Besides the window intiation, column and widget setup, here is what happens:

1. A global variable is defined to keep track across internal functions of the lastly selected item. This is to avoid the widgets emptying out whenever the user selects something else in any other window. It is kept empty for now.

```python
global last_item_selection
last_item_selection = None
```
2. The search bar is set up
```python
search = tk.StringVar()
def search_focus_in(event):
    """
    Triggered when user selects the searchbar.
    Removes placeholder text and sets font color to black.
    """
    if search_box.get() == "Search...":
        search_box.delete(0, tk.END)
        search_box.config(foreground="black")
def search_focus_out(event):
    """
    Triggered when user de-selects the searchbar, and the searchbar is empty.
    Sets font color back to grey and adds placeholder text.
    """
    if search_box.get() == "":
        search_box.insert(0, "Search...")
        search_box.config(foreground="grey")
search_box = ttk.Entry(itemsframe, textvariable=search) # create widget
search_box.grid(row=1, column=0, sticky="we")
search_box.insert(0, 'Search...')
search_box.config(foreground="grey")
search_box.bind("<FocusIn>", search_focus_in)
search_box.bind("<FocusOut>", search_focus_out)
search_box.bind("<KeyRelease>", load_items) # refresh items list (load_items) whenever user types something in the searchbar
```


2. A function is defined to refresh the list of items based on the search query

```python
def load_items(event=None):
    """
    Refreshes item listbox and filters it based on user input in the searchbar.
    """
    # get current query from searchbar
    if search.get() == 'Search...':
        search_query = '' # placeholder text means no actual query
    else:
        search_query = search.get() # get content from searchbar

    items_list.delete(0, tk.END) # empty listbox
    cur.execute("SELECT * FROM items WHERE name LIKE ? AND name NOT LIKE '%DELETED%' ORDER BY name", ('%' + search_query + '%',))   # get list of non-deleted items whose name matches the query
    items = cur.fetchall()
    for row in items:
        text = f"{row[1]} | (ID: {row[0]})" # name and ID, respecively
        items_list.insert(tk.END, text) # insert item content in the listbox
```


3. Another function refreshes the label widgets to reflect the fields of the currently selected item
```python
def load_item_data(...):
    
    # check if item is selected, if not fill labels with 'n/a' and 'No item selected yet' as the name
    
    # if something is selected:
    
        # retrieve item data with get_item_data()
        
        # update label widgets accordingly
```


4. Another function gets called when user clicks 'edit' next to a field:

```python
def items_edit_button(...)
    
    # if nothing is selected or the selected item has been deleted, interrupt
    
    # open popup window
    
    # set up labels to display current value and provide user with an entry field
    
    # define a function to be called when user clicks 'Confirm'. It replaces the old value with the new value provided by the user and logs the event.
```
![edit_field](notebook_images/edit_field.png)


5. Function for editing the owner / location fields:

```python
def items_edit_person_button(...):
    
    # same as previous function, but with a list of all people to select one from, instead of an entry field. The listbox + searchbar setup is the same as above.
```
![edit_location](notebook_images/edit_location.png)


6. Function for when user clicks 'Add item'

```python
def items_add_button(...)

    # ONLY IF an empty item doesn't exist already, create one and log event

    # unselect current item
    
    # select empty item and highlight it
    
    # scroll to empty item
    
    # refresh widgets
```


7. Function for 'Delete item' button

```python
def items_remove_button(...)

    # if no item is selected, interrupt 
    
    # if item is already deleted, show warning message telling user to refresh the menu manually
    
    # show popup message to confirm deletion
    
    # delete item and log event

```

## The people menu
![peoplemenu](notebook_images/peoplemenu.png)

This is where I started realising that dividing everything into **separate windows** might not have been a smart choice. At this point I already worked a lot on making the items menu work, and did not have the foresight to make the window setup and its functions **more 'pythonically' replicable** than just copy-and-pasting. But alas, I could not start from scratch at this point of the project and decided to go for a copy-and-paste solution.

The only change was a more complex for loop of SQL queries to have a list of all the items owned / borrowed by a person show up in its label. The owned / borrowed items are not editable because it would have required a much more complex popup. Instead, it was reasonable to expect the user to edit item ownership / location only via the items menu.

## The events log (History of changes)
![logsmenu](notebook_images/logsmenu.png)

Again, here the code is unfortunately mostly copy-and-pasted from earlier listbox & searchbar setups, with minor edits to adjust the window size and display of entries. Without making the code even more convoluted, I could only have the user search either an event's time or description. I opted for the latter, as the events are already displayed in chronological order.

## Final thoughts
If I could go back, **I would have definetly gone about this project differently**. I would have kept everything (exept logs and warning popups) in a single window to make it easier to modularly load and implement functions. This would have probably halved the lines of code. But I have this remorse only because **I am now more aquainted with tkinter**, which only happened because I learned as I worked my way through this project.

In the end, I'm satisfied that **the UI works as intended** with only **a single hiccup** (the selected item changing if its name is edited, since the alphabetical position of the item in the list changes). I made sure the code would not break in all other circumstances I could foresee. I could very well see this being used in a small-scale, self-managed, community project.